In [363]:
import generator

import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})

from skopt.plots import plot_gaussian_process
from skopt import Optimizer

In [364]:
bins_file = open('../exp_data/na49/binning.bin', 'r')

In [365]:
bins = []

for num, line in enumerate(bins_file.readlines()):

    if num % 2 == 0:
        
        a = line.split('|')

        for bin in a:

            bin = bin.replace('(', '').replace(')', '').replace(' ', '')
            bins.append(list(map(float, bin.split(','))))

bins = np.array(bins)

bins_file.close()

In [366]:
import pandas as pd

ids = [211]

refs = {}

for id in ids:
    
    refs[id] = pd.read_csv(f'../exp_data/na49/{id}.csv')

In [367]:
inst = {
    'Print:quiet': 'on',

    'Beams:frameType': '2',

    'Beams:idA': '2212',
    'Beams:idB': '2212',

    'Beams:eA': '158',
    'Beams:eB': '0',

    'SoftQCD:all': 'on',

    'Tune:pp': '1',

    'SigmaDiffractive:dampen': 'on',
    'SpaceShower:phiIntAsym': 'on',
    'SpaceShower:phiPolAsym': 'on',
    'SpaceShower:rapidityOrder': 'on',
    'SpaceShower:rapidityOrderMPI': 'on',
    'SpaceShower:samePTasMPI': 'off',
    'TimeShower:dampenBeamRecoil': 'on',
    'TimeShower:phiPolAsym': 'on',

    'MultipartonInteractions:ecmRef': '1800',
    }

In [398]:
optim_param = 'StringPT:sigma'

In [369]:
def objective(x, n_events, refs, agregate=None):

    new_val = x[0]

    inst[optim_param] = f'{new_val}'

    data, sigma = generator.generate(n_events, inst, bins)
    res =  generator.get_score(data, sigma, n_events, bins, refs, agregate)

    return res

In [370]:
n_initial_points = 1

opt_gp = Optimizer([(0., 1.)],
                   base_estimator="GP",
                   n_initial_points=n_initial_points,
                   acq_optimizer="sampling", acq_func='EI',
                   random_state=42, n_jobs=-1)

In [371]:
n_events = int(1e6)

itrs = 20

verbose = 1

zero = '0'

plt.figure(figsize=(5, 5))

for i in range(itrs):

    if i % verbose == 0:
        print(f'Iteration number {i + 1}')
    
    next_x = opt_gp.ask()
    f_val = objective(next_x, n_events, refs, np.sum)
    res = opt_gp.tell(next_x, f_val)
    
    ax1 = plot_gaussian_process(res,
        #noise_level=noise_level,
        show_legend=True, show_title=True,
        show_next_point=False, show_acq_func=False
        )
    
    ax1.legend(loc='upper right')
    ax1.set_title('')

    ax1.set_ylim((50, 1150))
    ax1.set_ylabel('$f$')
    ax1.set_xlabel(optim_param)

    ax1.figure.savefig(f'opt_res/{optim_param}_f_{zero * int(np.trunc(np.log10(itrs - i)))}{i}.png',
                      pad_inches=0.5, bbox_inches='tight')
    ax1.clear()
    
    ax2 = plot_gaussian_process(res,
                               #noise_level=noise_level,
                               show_legend=True, show_title=False,
                               show_next_point=True, show_acq_func=True,
                               show_observations=False,
                               show_mu=False)
    
    ax2.legend(loc='upper right')
    ax2.set_title('')

    ax2.set_yscale('log')
    ax2.set_ylim((1e-50, 1e2))
    ax2.set_ylabel('EI')
    ax2.set_xlabel(optim_param)
    ax2.figure.savefig(f'opt_res/{optim_param}_ei_{zero * int(np.trunc(np.log10(itrs - i)))}{i}.png',
                      pad_inches=0.5, bbox_inches='tight')
    ax2.clear()

plt.close()

f_min = np.min(opt_gp.yi)
x_min = opt_gp.Xi[np.argmin(opt_gp.yi)]

print(f'f_min = {f_min:.4} at x_min = {x_min[0]:.4}')

Iteration number 1


100%|██████████| 1000000/1000000 [02:09<00:00, 7725.15it/s]


Iteration number 2


100%|██████████| 1000000/1000000 [02:22<00:00, 7000.90it/s]


Iteration number 3


...

Iteration number 19


100%|██████████| 1000000/1000000 [02:10<00:00, 7657.45it/s]


Iteration number 20


100%|██████████| 1000000/1000000 [02:11<00:00, 7623.87it/s]


f_min = 84.97 at x_min = 0.3692


In [376]:
import glob
import contextlib
from PIL import Image

def make_animation(fp_in, fp_out):

    # use exit stack to automatically close opened images
    with contextlib.ExitStack() as stack:

        # lazily load images
        imgs = (stack.enter_context(Image.open(f))
                for f in sorted(glob.glob(fp_in)))

        # extract  first image from iterator
        img = next(imgs)

        # https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
        img.save(fp=fp_out, format='GIF', append_images=imgs,
                save_all=True, duration=400, loop=0)

In [399]:
make_animation(f'opt_res/{optim_param}_f*.png',
               f'opt_res/{optim_param}_f.gif')
make_animation(f'opt_res/{optim_param}_ei*.png',
               f'opt_res/{optim_param}_ei.gif')

In [393]:
optim_param = 'MultipartonInteractions:expPow'

In [394]:
n_initial_points = 1

opt_gp = Optimizer([(0.4, 10.)],
                   base_estimator="GP",
                   n_initial_points=n_initial_points,
                   acq_optimizer="sampling", acq_func='EI',
                   random_state=42, n_jobs=-1)

In [395]:
n_events = int(1e6)

itrs = 20

verbose = 1

zero = '0'

plt.figure(figsize=(5, 5))

for i in range(itrs):

    if i % verbose == 0:
        print(f'Iteration number {i + 1}')
    
    next_x = opt_gp.ask()
    f_val = objective(next_x, n_events, refs, np.sum)
    res = opt_gp.tell(next_x, f_val)
    
    ax1 = plot_gaussian_process(res,
        #noise_level=noise_level,
        show_legend=True, show_title=True,
        show_next_point=False, show_acq_func=False
        )
    
    ax1.legend(loc='upper right')
    ax1.set_title('')

    ax1.set_ylim((80, 130))
    ax1.set_ylabel('$f$')
    ax1.set_xlabel(optim_param)

    ax1.figure.savefig(f'opt_res/{optim_param}_f_{zero * int(np.trunc(np.log10(itrs - i)))}{i}.png',
                      pad_inches=0.5, bbox_inches='tight')
    ax1.clear()
    
    ax2 = plot_gaussian_process(res,
                               #noise_level=noise_level,
                               show_legend=True, show_title=False,
                               show_next_point=True, show_acq_func=True,
                               show_observations=False,
                               show_mu=False)
    
    ax2.legend(loc='upper right')
    ax2.set_title('')

    ax2.set_yscale('log')
    ax2.set_ylim((1e-50, 1e2))
    ax2.set_ylabel('EI')
    ax2.set_xlabel(optim_param)
    ax2.figure.savefig(f'opt_res/{optim_param}_ei_{zero * int(np.trunc(np.log10(itrs - i)))}{i}.png',
                      pad_inches=0.5, bbox_inches='tight')
    ax2.clear()

plt.close()

f_min = np.min(opt_gp.yi)
x_min = opt_gp.Xi[np.argmin(opt_gp.yi)]

print(f'f_min = {f_min:.4} at x_min = {x_min[0]:.4}')

Iteration number 1


100%|██████████| 1000000/1000000 [02:06<00:00, 7931.75it/s]


Iteration number 2


100%|██████████| 1000000/1000000 [02:06<00:00, 7904.74it/s]


Iteration number 3


...

Iteration number 19


100%|██████████| 1000000/1000000 [02:09<00:00, 7707.80it/s]


Iteration number 20


100%|██████████| 1000000/1000000 [02:11<00:00, 7608.94it/s]


f_min = 84.82 at x_min = 5.252


In [397]:
make_animation(f'opt_res/{optim_param}_f*.png',
               f'opt_res/{optim_param}_f.gif')
make_animation(f'opt_res/{optim_param}_ei*.png',
               f'opt_res/{optim_param}_ei.gif')